In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Concatenate, TimeDistributed, RepeatVector
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping

# 데이터 불러오기
df = pd.read_csv('../../Data/using/dt_2.6/ByAstar_dt_2.6_1016.csv')

df = df.dropna(subset=['combination'])

# 데이터셋 분리: train_set 8 : test_set 2
train_df, test_df = train_test_split(df, shuffle=True, test_size=0.2)
# train_set을 다시 8:2로 나눠서 train_set과 validation_set을 만듦
train_df, val_df = train_test_split(train_df, shuffle=True, test_size=0.2)
# random_state=42

# 모든 시퀀스의 길이 중에서 최대 길이를 구하기
all_sequences = [eval(str(seq)) for seq in df['combination'].values]
max_seq_length = max([len(seq) for seq in all_sequences])

# 각 데이터셋에서 theta, phi, sequence 추출하고 reshape 적용
theta_train = train_df['Theta'].values.reshape(-1, 1)
phi_train = train_df['Phi'].values.reshape(-1, 1)
sequence_train = pad_sequences(train_df['combination'].apply(eval).tolist(), maxlen=max_seq_length, padding='pre')

theta_val = val_df['Theta'].values.reshape(-1, 1)
phi_val = val_df['Phi'].values.reshape(-1, 1)
sequence_val = pad_sequences(val_df['combination'].apply(eval).tolist(), maxlen=max_seq_length, padding='pre')

theta_test = test_df['Theta'].values.reshape(-1, 1)
phi_test = test_df['Phi'].values.reshape(-1, 1)
sequence_test = pad_sequences(test_df['combination'].apply(eval).tolist(), maxlen=max_seq_length, padding='pre')


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

def create_model(hidden_units=128, dropout_rate=0.0, learning_rate=0.001):
    theta_input = Input(shape=(1,), name='theta_input')
    phi_input = Input(shape=(1,), name='phi_input')
    merged = Concatenate()([theta_input, phi_input])
    repeated_vector = RepeatVector(max_seq_length)(merged)
    
    rnn_layer = GRU(hidden_units, return_sequences=True, name='rnn_layer')(repeated_vector)
    rnn_layer = Dropout(dropout_rate)(rnn_layer)
    rnn_layer2 = GRU(hidden_units, return_sequences=True, name='rnn_layer2')(rnn_layer)
    rnn_layer2 = Dropout(dropout_rate)(rnn_layer2)
    rnn_layer3 = GRU(hidden_units, return_sequences=True, name='rnn_layer3')(rnn_layer2)
    output = TimeDistributed(Dense(5, activation='softmax'), name='output_layer')(rnn_layer3)
    
    model = Model(inputs=[theta_input, phi_input], outputs=output)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=learning_rate), metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=10000, batch_size=64, verbose=0)
param_distributions = {
    'hidden_units': [64, 128, 256],
    'dropout_rate': [0.0, 0.1, 0.2, 0.3],
    'learning_rate': [0.001, 0.01, 0.1]
}

random_search = RandomizedSearchCV(estimator=model, param_distributions=param_distributions, n_iter=10, cv=3, verbose=2)
random_search_results = random_search.fit([theta_train, phi_train], np.expand_dims(sequence_train, -1))
print("Best Score: ", random_search_results.best_score_)
print("Best Params: ", random_search_results.best_params_)
